In [1]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
from sklearn.ensemble import RandomForestRegressor
import sklearn.preprocessing as preprocessing
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve, cross_val_score, train_test_split

# 1. Data Pre-processing

### Fill NaN

In [2]:
train_raw_data = pd.read_csv("raw_data/train.csv",parse_dates=['purchase_date','release_date'])
train_raw_data['purchase_date'] = train_raw_data.purchase_date.fillna(method='backfill')
train_raw_data['total_positive_reviews'] = train_raw_data.total_positive_reviews.fillna(method='backfill')
train_raw_data['total_negative_reviews'] = train_raw_data.total_negative_reviews.fillna(method='backfill')

In [3]:
test_raw_data = pd.read_csv("raw_data/test.csv",parse_dates=['purchase_date','release_date'])
test_raw_data['purchase_date'] = test_raw_data.purchase_date.fillna(method='backfill')
test_raw_data['total_positive_reviews'] = test_raw_data.total_positive_reviews.fillna(method='backfill')
test_raw_data['total_negative_reviews'] = test_raw_data.total_negative_reviews.fillna(method='backfill')

# 2. Feature Engineering 

### 2.1 Extract date feature

In [4]:
# define date-extracting function
def extract_date(df_copy,df,column):
    df_copy[column+'_year']=df[column].apply(lambda x: x.year)
    df_copy[column+'_month']=df[column].apply(lambda x: x.month)
    df_copy[column+'_day']=df[column].apply(lambda x: x.day)
    return df_copy

In [5]:
# for training set
# extract purchase_date
train_raw_copy = train_raw_data.copy()
train_extract_purchase_date = extract_date(train_raw_copy, train_raw_data, 'purchase_date')

# extract release_date
train_extract_purchase_date_copy = train_extract_purchase_date.copy()
train_extract_date = extract_date(train_extract_purchase_date_copy, train_extract_purchase_date, 'release_date')

import datetime
train_extract_date['date_interval'] = (train_extract_date['purchase_date']-train_extract_date['release_date'])\
.apply(lambda x: x.days)

del train_raw_copy,train_extract_purchase_date,train_extract_purchase_date_copy

In [6]:
# for test set
# extract purchase_date
test_raw_copy = test_raw_data.copy()
test_extract_purchase_date = extract_date(test_raw_copy, test_raw_data, 'purchase_date')

# extract release_date
test_extract_purchase_date_copy = test_extract_purchase_date.copy()
test_extract_date = extract_date(test_extract_purchase_date_copy, test_extract_purchase_date, 'release_date')

import datetime
test_extract_date['date_interval'] = (test_extract_date['purchase_date']-test_extract_date['release_date'])\
.apply(lambda x: x.days)

del test_raw_copy,test_extract_purchase_date,test_extract_purchase_date_copy

### 2.2 Extract category feature

In [7]:
train_categories_one_hot = train_raw_data["categories"].str.get_dummies(",") 
test_categories_one_hot = test_raw_data["categories"].str.get_dummies(",") 
categories_train_diff_test = train_categories_one_hot.columns.difference(test_categories_one_hot.columns)
categories_test_diff_train = test_categories_one_hot.columns.difference(train_categories_one_hot.columns)
print(categories_train_diff_test)
print(categories_test_diff_train)

Index(['Valve Anti-Cheat enabled'], dtype='object')
Index([], dtype='object')


In [8]:
train_categories_one_hot = train_categories_one_hot.drop(columns=list(categories_train_diff_test),axis=1)

### 2.3 Extract genre feature

In [9]:
train_genres_one_hot = train_raw_data["genres"].str.get_dummies(",") 
test_genres_one_hot = test_raw_data["genres"].str.get_dummies(",") 
genres_train_diff_test = train_genres_one_hot.columns.difference(test_genres_one_hot.columns)
genres_test_diff_train = test_genres_one_hot.columns.difference(train_genres_one_hot.columns)
print(genres_train_diff_test)
print(genres_test_diff_train)

Index(['Animation & Modeling', 'Audio Production', 'Design & Illustration',
       'Racing', 'Sexual Content', 'Utilities'],
      dtype='object')
Index([], dtype='object')


In [10]:
train_genres_one_hot = train_genres_one_hot.drop(columns=list(genres_train_diff_test),axis=1)

### 2.4 Extract tag feature

In [11]:
train_tags_one_hot = train_raw_data["tags"].str.get_dummies(",") 
test_tags_one_hot = test_raw_data["tags"].str.get_dummies(",") 
tags_train_diff_test = train_tags_one_hot.columns.difference(test_tags_one_hot.columns)
tags_test_diff_train = test_tags_one_hot.columns.difference(train_tags_one_hot.columns)
print(tags_train_diff_test)
print(tags_test_diff_train)

Index(['3D', 'ATV', 'Addictive', 'Animation & Modeling',
       'Artificial Intelligence', 'Audio Production', 'Automation', 'Batman',
       'Battle Royale', 'Bikes', 'Board Game', 'Bullet Hell', 'Capitalism',
       'Card Game', 'Cartoon', 'Cats', 'Character Action Game', 'Chess',
       'Choose Your Own Adventure', 'Clicker', 'Co-op Campaign', 'Comic Book',
       'Conspiracy', 'Dark Comedy', 'Design & Illustration', 'Documentary',
       'Dungeons & Dragons', 'Experience', 'Flight', 'Game Development',
       'God Game', 'Gothic', 'Gun Customization', 'Hidden Object', 'Horses',
       'Immersive Sim', 'Intentionally Awkward Controls', 'Investigation',
       'LGBTQ+', 'Lara Croft', 'Logic', 'MMORPG', 'Mars', 'Martial Arts',
       'Metroidvania', 'Motocross', 'Motorbike', 'Movie', 'Multiple Endings',
       'Naval', 'Offroad', 'Perma Death', 'Pirates', 'Programming', 'PvE',
       'Quick-Time Events', 'Racing', 'Rome', 'Sailing', 'Satire',
       'Score Attack', 'Sequel', 'Sniper',

In [12]:
train_tags_one_hot = train_tags_one_hot.drop(columns=list(tags_train_diff_test),axis=1)
test_tags_one_hot = test_tags_one_hot.drop(columns=list(tags_test_diff_train),axis=1)

### 2.5 Feature Concat 

In [13]:
train = pd.concat([train_extract_date, train_categories_one_hot,train_genres_one_hot,train_tags_one_hot],axis=1)
test = pd.concat([test_extract_date, test_categories_one_hot,test_genres_one_hot,test_tags_one_hot],axis=1)
print('train shape',train.shape)
print('test shape',test.shape)

train shape (357, 284)
test shape (90, 283)


### 2.6 Process repeated columns between tags, categories and genres

In [14]:
# for training set
set_tag = set(train_tags_one_hot.columns)
set_genre = set(train_genres_one_hot.columns)
set_category = set(train_categories_one_hot.columns)

set_1 = set_tag & set_genre
set_2 = set_category & set_tag
set_3 = set_category & set_genre
set_all = set_1 | set_2 | set_3
print('repeated columns：',set_all)
for col in set_all:
    tmp = train[[col]]
    tmp.columns=[col+'_1',col+'_2']
    tmp[col]=0
    tmp.loc[(tmp[col+'_1']==1) | (tmp[col+'_2']==1),[col]]=1
    train = train.drop([col],axis=1)
    tmp = tmp.drop([col+'_1',col+'_2'],axis=1)
    train = pd.concat([train,tmp],axis=1)
del set_1,set_2,set_3,set_all,tmp

repeated columns： {'Casual', 'Strategy', 'Nudity', 'Massively Multiplayer', 'Indie', 'Violent', 'Adventure', 'RPG', 'Simulation', 'Early Access', 'Free to Play', 'Co-op', 'Sports', 'Gore', 'Action'}


f:\python35\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
f:\python35\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
f:\python35\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [15]:
# for test set
set_tag = set(test_tags_one_hot.columns)
set_genre = set(test_genres_one_hot.columns)
set_category = set(test_categories_one_hot.columns)
set_1 = set_tag & set_genre
set_2 = set_category & set_tag
set_3 = set_category & set_genre
set_all = set_1 | set_2 | set_3
print('repeated columns：',set_all)
for col in set_all:
    tmp = test[[col]]
    tmp.columns=[col+'_1',col+'_2']
    tmp[col]=0
    tmp.loc[(tmp[col+'_1']==1) | (tmp[col+'_2']==1),[col]]=1
    test = test.drop([col],axis=1)
    tmp = tmp.drop([col+'_1',col+'_2'],axis=1)
    test = pd.concat([test,tmp],axis=1)
del set_1,set_2,set_3,set_all,tmp

repeated columns： {'Casual', 'Strategy', 'Nudity', 'Massively Multiplayer', 'Indie', 'Violent', 'Adventure', 'RPG', 'Simulation', 'Early Access', 'Free to Play', 'Co-op', 'Sports', 'Gore', 'Action'}


f:\python35\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
f:\python35\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
print('train shape',train.shape)
print('test shape',test.shape)

train shape (357, 269)
test shape (90, 268)


# 3. Regression：predict play time

### 3.1 Training (using all features)

In [17]:
train_input = train.drop(['categories','genres','tags','purchase_date','release_date','Free to Play'],axis=1)
train_x = train_input.drop(['playtime_forever','id'],axis=1)
train_y = train_input[['playtime_forever']]
print('train_x shape',train_x.shape)

train_x shape (357, 261)


In [18]:
####model####
from sklearn import ensemble
model = ensemble.GradientBoostingRegressor()
model.fit(train_x,train_y)

from sklearn.model_selection import cross_val_score
score = np.sqrt(-cross_val_score(model, train_x, train_y, cv=50,scoring='neg_mean_squared_error'))
mean_score = np.mean(score)
print(score)
print('mean_score:',mean_score)
del score,mean_score

f:\python35\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
f:\python35\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
f:\python35\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
f:\python35\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

f:\python35\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
f:\python35\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
f:\python35\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
f:\python35\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

[ 8.19597283  3.71923101  4.06795263  5.11958314 15.12714818  3.60396396
  4.6639295   1.06866697  3.26185174  2.36401532  2.88685344  9.5906575
  1.70971858  7.94773003  1.03001332  1.509885    4.58808453  1.58614015
  2.40872918  2.00857429  0.72261854  5.14726187 19.83656611  6.53547843
 10.21036191  4.34407221  1.33479096 33.45248868 15.38441465 10.82606113
 15.06584942  5.68304871 11.06149012  5.597549    1.66457078  2.05009459
  3.41625933 23.63640243 41.42816207 17.58115549  0.74224745  1.55849618
  2.05128604 12.16769565  1.46858326  0.9726531   1.00633027  1.88488292
 22.43727613  1.29111815]
mean_score: 7.340359337645156


f:\python35\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### 3.2 regression prediction

In [19]:
test_x = test.drop(['categories','genres','tags','purchase_date','release_date','id','Free to Play'],axis=1)
test_x = test_x[train_x.columns]  
print('test_x shape:',test_x.shape)

test_x shape: (90, 261)


In [20]:
test_y_pred = model.predict(test_x)
test["playtime_forever"] = test_y_pred
result=test[['id','playtime_forever']]
result.sort_values("playtime_forever",inplace=False)

,id,playtime_forever
16,16,-0.973445
48,48,-0.753384
3,3,-0.510598
2,2,-0.298963
51,51,-0.268618
55,55,-0.159363
70,70,-0.158034
67,67,-0.154186
26,26,-0.148159
1,1,0.016817


# 4. Store final result

In [21]:
result.loc[result['playtime_forever']<=0,['playtime_forever']] =0
result.sort_values("playtime_forever",inplace=False)

f:\python35\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,id,playtime_forever
48,48,0.000000
70,70,0.000000
67,67,0.000000
16,16,0.000000
55,55,0.000000
26,26,0.000000
2,2,0.000000
51,51,0.000000
3,3,0.000000
1,1,0.016817


In [22]:
result.to_csv('result/submit_1130_4_1.csv',index=0,header=1)